In [32]:

import pandas
from keras.models import Sequential
from keras.layers import Dense

# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [33]:

dataframe = pandas.read_csv("persistence/training.csv")
dataset = dataframe.drop('awardedMatches', axis=1).drop('id', axis=1).values
X = dataset[:,0:98].astype(float)
y = dataset[:, 98:].astype(int)
st_x= MinMaxScaler()
X= st_x.fit_transform(X)
print(X.shape, y.shape)

(65, 98) (65, 3)


In [34]:

# define the model
model = Sequential()
model.add(Dense(20, input_dim=98, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(X, y, epochs=150, batch_size=10)

Epoch 1/150
7/7 [==============================] - 2s 3ms/step - loss: 0.6535
Epoch 2/150
7/7 [==============================] - 0s 3ms/step - loss: 0.6192
Epoch 3/150
7/7 [==============================] - 0s 3ms/step - loss: 0.6125
Epoch 4/150
7/7 [==============================] - 0s 3ms/step - loss: 0.6062
Epoch 5/150
7/7 [==============================] - 0s 5ms/step - loss: 0.6012
Epoch 6/150
7/7 [==============================] - 0s 3ms/step - loss: 0.5961
Epoch 7/150
7/7 [==============================] - 0s 3ms/step - loss: 0.5927
Epoch 8/150
7/7 [==============================] - 0s

In [35]:

...
results = list()
n_inputs, n_outputs = X.shape[1], y.shape[1]
# define evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# enumerate folds
for train_ix, test_ix in cv.split(X):
	# prepare data
	X_train, X_test = X[train_ix], X[test_ix]
	y_train, y_test = y[train_ix], y[test_ix]
	# st_x= MinMaxScaler()
	# X_train= st_x.fit_transform(X_train)
	# X_test= st_x.fit_transform(X_test)
	# fit model
	model.fit(X_train, y_train, verbose=0, epochs=100)
	# make a prediction on the test set
	yhat = model.predict(X_test)
	print(yhat)
	print(y_test)
	# round probabilities to class labels
	yhat = yhat.round()
	# calculate accuracy
	acc = accuracy_score(y_test, yhat)
	# store result
	print('>%.2f' % acc)
	results.append(acc)

[[0.3076911  0.3425498  0.2548859 ]
 [0.21556205 0.13693416 0.4861224 ]
 [0.02787307 0.15001458 0.93024945]
 [0.18123513 0.11491343 0.67723703]
 [0.60820156 0.3589263  0.03489682]
 [0.06882811 0.3274269  0.48192632]
 [0.91766405 0.12859547 0.01775673]]
[[0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [1 0 0]]
>0.43
[[6.75479889e-01 8.64237666e-01 1.82598829e-04]
 [8.69559705e-01 2.39107549e-01 9.91111994e-03]
 [1.70905501e-01 8.54933381e-01 6.59164786e-03]
 [8.11712146e-01 2.25471199e-01 8.52340460e-03]
 [8.16948414e-01 5.36469519e-02 9.52160358e-02]
 [3.87144744e-01 8.23065042e-02 3.51609468e-01]
 [3.71185005e-01 1.13797754e-01 3.28682184e-01]]
[[1 0 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]]
>0.57
[[0.32098317 0.06024253 0.52942324]
 [0.70534045 0.04974917 0.16364485]
 [0.7996421  0.15983409 0.02909252]
 [0.3014832  0.73104805 0.01361722]
 [0.71513605 0.10710186 0.08088535]
 [0.04277453 0.10602242 0.8356284 ]
 [0.55134106 0.06339991 0.2932248 ]]
[[0 0 1]
 [1 0 0]

In [36]:
print('Average accuracy: %.2f%% \nStandart deviation: %.2f%%' % ((mean(results))*100, (std(results)*100)))

Average accuracy: 93.10% 
Standart deviation: 13.32%
